# 文本相似度实例（双塔模型）

## Step1 导入相关包

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

## Step3 划分数据集

## Step4 数据集预处理

## Step5 创建模型

## Step6 创建评估函数

## Step7 创建TrainingArguments

## Step8 创建Trainer

## Step9 模型训练

## Step10 模型评估

## Step11 模型预测